## Классификация текстов с использованием предобученных языковых моделей.

В данном задании вам предстоит обратиться к задаче классификации текстов и решить ее с использованием предобученной модели BERT.

[Статья на хабр](https://habr.com/ru/articles/498144/)

In [ ]:
import json
# do not change the code in the block below
# __________start of block__________
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline
# __________end of block__________

In [ ]:
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Обратимся к набору данных SST-2. Holdout часть данных (которая понадобится вам для посылки) доступна по ссылке ниже.

In [ ]:
# do not change the code in the block below
# __________start of block__________

# !wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
# __________end of block__________

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# do not change the code in the block below
# __________start of block__________
df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True))) # сначала все кодируем

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

texts_train = padded[:5000, :]
# texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]

texts_test = padded[5000:, :]
# texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]

with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)
    texts_holdout = pd.Series(texts_holdout).apply((lambda x : tokenizer.encode(x, add_special_tokens=True)))
    max_len = 0
    for i in texts_holdout.values:
        if len(i) > max_len:
            max_len = len(i)
    padded_holdout = np.array([i + [0]*(max_len-len(i)) for i in texts_holdout.values])
# __________end of block__________

Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [ ]:
attention_mask = np.where(texts_train != 0, 1, 0) # создаем attention-маску
attention_mask.shape

(5000, 67)

In [ ]:
texts_test.shape

(1920, 67)

In [ ]:
attention_mask2 = np.where(texts_test != 0, 1, 0) # создаем attention-маску
attention_mask2.shape

(1920, 67)

In [ ]:
train_ids = torch.tensor(np.array(texts_train))
attention_mask = torch.tensor(attention_mask)
with torch.no_grad():
    last_hidden_states = model(train_ids, attention_mask=attention_mask)

In [ ]:
test_ids = torch.tensor(np.array(texts_test))
attention_mask2 = torch.tensor(attention_mask2)
with torch.no_grad():
    last_hidden_states1 = model(test_ids, attention_mask=attention_mask2)

In [ ]:
features_train = last_hidden_states[0][:,0,:].numpy()
features_test = last_hidden_states1[0][:,0,:].numpy()

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(features_train, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

In [ ]:
print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.8328


In [ ]:
lr_clf = LogisticRegression(C=5.263252631578947)
lr_clf.fit(features_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=5.263252631578947)

In [ ]:
attention_mask3 = np.where(padded_holdout != 0, 1, 0) # создаем attention-маску
attention_mask.shape


holdout_ids = torch.tensor(np.array(padded_holdout))
attention_mask3 = torch.tensor(attention_mask3)
with torch.no_grad():
    last_hidden_states3 = model(holdout_ids, attention_mask=attention_mask3)

In [ ]:
features_holdout = last_hidden_states3[0][:,0,:].numpy()

In [ ]:
# from sklearn.metrics import accuracy
roc_auc_score(y_train, out_dict['train'])

0.9582449514318309

#### Сдача задания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к первому (положительному) классу

In [ ]:
out_dict = {
    'train': list(grid_search.predict_proba(features_train)[:, 1]),# list of length 5000 with probas
    'test': list(grid_search.predict_proba(features_test)[:, 1]),# list of length 1920 with probas
    'holdout': list(grid_search.predict_proba(features_holdout)[:, 1])# list of length 500 with probas
}

Несколько `assert`'ов для проверки вашей посылки:

In [ ]:
out_dict

{'train': [0.9896268361230396,
  0.0011220137124959765,
  0.02082338921853827,
  0.9274077054064734,
  0.14845575804163416,
  0.4575164688527213,
  0.024060231752945346,
  0.9977127912781547,
  0.03427458587321096,
  0.19334766052013963,
  0.995334193921453,
  0.09721884510401074,
  0.15532887663526887,
  0.9990660601785175,
  0.45866050501280314,
  0.4502768918594199,
  0.04674452137098903,
  0.993780383875017,
  0.46891668995540614,
  0.54389754579641,
  0.07289803524896564,
  0.931080007346491,
  0.22927402841864544,
  0.22551686864469025,
  0.00039233415491602244,
  0.9831317357938478,
  0.9915561568059017,
  0.03760480202026625,
  0.9975967828513971,
  0.8875279335111764,
  0.6808423602617355,
  0.9643001341807554,
  0.9917363181142006,
  0.6760161257150157,
  0.26866826027204427,
  0.12380781208451365,
  0.0044382546632464265,
  0.28197922122664815,
  0.9914692443141112,
  0.729991701438472,
  0.23434492433404916,
  0.5185867758352768,
  0.6478560939763248,
  0.000493569877161855

In [ ]:
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["train"]) == 5000
), "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["test"]) == 1920
), "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert len(
    out_dict["holdout"]
)  == 500, "The predicted probas list length does not match the holdout set size"

Запустите код ниже для генерации посылки.

In [ ]:
# do not change the code in the block below
# __________start of block__________
FILENAME = "submission_dict_hw_text_classification_with_bert.json"

with open(FILENAME, "w") as iofile:
    json.dump(out_dict, iofile)
print(f"File saved to `{FILENAME}`")
# __________end of block__________

File saved to `submission_dict_hw_text_classification_with_bert.json`


На этом задание завершено. Поздравляем!